In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the data
(x_train, y_train), (x_test, y_test) = keras.datasets.load_dir("dataset")


In [4]:
# Define the dimensions of the input image
input_shape = (1300, 1300, 3)

# Encoder
encoder_input = layers.Input(shape=input_shape)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoder_output = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Create the autoencoder model
autoencoder = keras.Model(encoder_input, decoder_output)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
# fit the model with early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
autoencoder.fit(x_train, x_train, epochs=100, batch_size=128, shuffle=True, validation_data=(x_test, x_test), callbacks=[early_stopping])


In [5]:

# Saving entire autoencoder model
autoencoder.save('autoencoder.h5')

# Saving encoder models so that we can use it to compress the images
encoder_input.save('encoder_input.h5')

# Saving decoder models so that we can use it to decompress the images
decoder_output.save('decoder_output.h5')

In [8]:
folder_path = 'data/dataset/NoisyImages'
folder_path_2 = 'data/dataset/ProcessedImages'
import os
from PIL import Image

# Initialize an empty list to hold the images
images = []

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.tif'):  # You can add more formats if needed
        img_path = os.path.join(folder_path, filename)
        # Open and resize the image
        with Image.open(img_path) as img:
            img = img.resize((1300, 1300))  # Resize to 1300x1300
            img_array = np.array(img)  # Convert to NumPy array
            images.append(img_array)  # Add the image to the list

# Convert the list of images to a NumPy array
x_train = np.array(images)

images_2 = []

for filename in os.listdir(folder_path_2):
    if filename.endswith('.tif'):  # You can add more formats if needed
        img_path = os.path.join(folder_path_2, filename)
        # Open and resize the image
        with Image.open(img_path) as img:
            img = img.resize((1300, 1300))  # Resize to 1300x1300
            img_array = np.array(img)  # Convert to NumPy array
            images_2.append(img_array)  # Add the image to the list

# Convert the list of images to a NumPy array
x_train = np.array(images)
y_train = np.array(images_2)

# Check the shape of the resulting array
print(x_train.shape)
print(y_train.shape)

(40, 1300, 1300, 3)
(40, 1300, 1300, 3)
